In [10]:
import cv2
import numpy as  np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Rescaling
from keras.models import Sequential

In [11]:
def crop_minAreaRect(rect, box, gray):
    mult = 1 
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv2.getRectSubPix(gray, size, center)    
    cropped = cv2.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    
    return croppedRotated

In [12]:
input_shape = (64, 64, 3)


model = Sequential()
model.add(Conv2D(16, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv1',input_shape=(64, 64, 3)))
model.add(Conv2D(16, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv1'))
model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv2'))
model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv3'))
model.add(Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv2'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv3'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv2'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv3'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv4'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 64, 64, 16)        448       
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 16)        2320      
                                                                 
 block1_maxpool (MaxPooling2  (None, 32, 32, 16)       0         
 D)                                                              
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       18560     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_maxpool (MaxPooling2  (None, 16, 16, 128)      0         
 D)                                                   

In [13]:
model = tf.keras.models.load_model("model_vgg19.h5")

In [14]:
from keras.preprocessing.image import img_to_array


In [15]:
img_crop=cv2.imread("91.jpg",1)
img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
img_crop = img_to_array(img_crop)
img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
Y_box_pred = model.predict(img_crop)
print(Y_box_pred)

[[3.7082053e-09]]


In [16]:
video_path='test2.mp4'
cap=cv2.VideoCapture(video_path)
backSub=cv2.createBackgroundSubtractorMOG2()
if(cap.isOpened()==False):
    print("Khong mo duoc video")
while (cap.isOpened()):
    ret, frame=cap.read()
    if ret==True:
        #Thuat toan tru nen
        fgMask=backSub.apply(frame)
        fgMask=cv2.cvtColor(fgMask, 0)
        #Dung bo loc ma tran 5*5
        kernel = np.ones((7,7), np.uint8)
        #Lay gia tri pixel nho nhat cua kernel lam manh net
        fgMask=cv2.erode(fgMask,kernel,iterations=1)
        #Lay gia tri pixel lon nhat cua kernel lam lien net
        fgMask=cv2.dilate(fgMask,kernel,iterations=1)
        
        #tao them su lien net bang ham open
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel,iterations=1)
        #lay nguong 120
        _,fgMask = cv2.threshold(fgMask,120,255,cv2.THRESH_BINARY)
        #phat hien canh bang candy
        fgMask=cv2.Canny(fgMask, 20, 200)
        contours,_ = cv2.findContours(fgMask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            rect = cv2.minAreaRect(contours[i])
            if (rect[1][0]*rect[1][1]) < 1000:
                continue
            
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            img_crop = crop_minAreaRect(rect, box, frame)       
            img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
            img_crop = img_to_array(img_crop)
            img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
            cv2.drawContours(frame,[box],0,(0,255,0),2)
        
        
            Y_box_pred = model.predict(img_crop)
            if Y_box_pred <= 0.5:
                cv2.putText(frame, "Person", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if Y_box_pred > 0.5:
                cv2.putText(frame, "Non", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cv2.destroyAllWindows()    
            